# 공공 데이터 API 기반 크롤링

## 1. 출입국관광통계서비스 - 개발 계정 받기

In [1]:
# Service_Enc_Key = "QYiQcd%2Biv7G6qpmPO2WR1Qo%2FrSghkwXi9Di3hA3nZTOH4d5RXgTRGLsHJ5aLg%2BVS0nDMAwotJ8SLFuBjWh8Mag%3D%3D"
Service_Enc_Key = "QYiQcd%2Biv7G6qpmPO2WR1Qo%2FrSghkwXi9Di3hA3nZTOH4d5RXgTRGLsHJ5aLg%2BVS0nDMAwotJ8SLFuBjWh8Mag%3D%3D"
# Service_Dec_Key = "QYiQcd+iv7G6qpmPO2WR1Qo/rSghkwXi9Di3hA3nZTOH4d5RXgTRGLsHJ5aLg+VS0nDMAwotJ8SLFuBjWh8Mag=="
Service_Dec_Key = "QYiQcd+iv7G6qpmPO2WR1Qo/rSghkwXi9Di3hA3nZTOH4d5RXgTRGLsHJ5aLg+VS0nDMAwotJ8SLFuBjWh8Mag=="

In [2]:
import requests

url = 'http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList'
# params ={'serviceKey' : '서비스키', 'YM' : '201201', 'NAT_CD' : '112', 'ED_CD' : 'E' }
params ={'serviceKey' : Service_Enc_Key, 'YM' : '201201', 'NAT_CD' : '112', 'ED_CD' : 'E' }

response = requests.get(url, params=params)
print(response.content)

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><responseTime>2024-03-13T09:54:07.856+09:00</responseTime><resultCode>30</resultCode><resultMsg>SERVICE KEY IS NOT REGISTERED ERROR.</resultMsg></header></response>'


## 2. 공공데이터 크롤링하기

- Open API를 이용하여 출입국관광통계서비스 데이터 중에서 2017년부터 현재까지 입국한 중국인의 수를 크롤링해 보자

### (1) 전체 작업 설계하기

#### 1. 데이터를 수집할 국가코드와 연도 입력하기

        - national_code, nStartYear, nEndYear

#### 2. 데이터 수집 요청하기: getTourismStatsService()

        - url 구성하여 데이터 요청하기: getTourismStatsItem()
        - url 접속하고 요청하기: getRequestUrl()
        - 응답 데이터를 리스트로 구성하기: jsonResult.result

#### 3. 데이터를 JSON 파일과 CSV 파일로 저장하기: json.dumps().to_csv()

In [3]:
import os
import sys
import urllib.request
import datetime
import timer
import json
import pandas as pd

# ServiceKey = Service_Dec_Key
ServiceKey="Ody77GLuYeR%2FeFqbpduMN2Bi4Cka2fztbgnj6E2Eux1kUhy3e4epR28XKBUaObiqPoVzAizxXMBPXtMyuC9v9Q%3D%3D"


def getRequestUrl(url):
    req = urllib.request.Request(url)
    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
        
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None
    
def getTourismStatsItem(yyyymm, nat_cd, ed_cd):
    service_url = "http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList"
    parameters = "?_type=json&serviceKey=" + ServiceKey # 인증키
    parameters += "&YM=" + yyyymm
    parameters += "&NAT_CD=" + nat_cd
    parameters += "&ED_CD=" + ed_cd

    url = service_url + parameters
    print(url)
    responseDecode = getRequestUrl(url)

    if(responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)
    
def getTourismStatsService(nat_cd, ed_cd, nStartYear, nEndYear):
    jsonResult = []
    result = []
    natName = ''
    dataEND = "{0}{1:0>2}".format(str(nEndYear), str(12))  # 데이터 끝 초기화
    isDataEnd = 0  # 데이터 끝 확인용 flag 초기화
    ed = '' # 오류 발생에 따른 수정 : local variable 'ed' referenced before assignment

    for year in range(nStartYear, nEndYear):
        for month in range(1, 13):
            if(isDataEnd == 1): break   # 데이터 끝 flag 설정되어 있으면 작업 중지
            yyyymm = "{0}{1:0>2}".format(str(year), str(month))
            jsonData = getTourismStatsItem(yyyymm, nat_cd, ed_cd)
            if (jsonData['response']['header']['resultMsg'] == 'OK'):
                # 입력된 범위까지 수집하지 않았지만, 더이상 제공되는 데이터가 없는 마지막 항목인 경우
                if jsonData['response']['body']['items'] == '':
                    isDataEnd = 1  # 데이터 끝 flag 설정
                    dataEND = '{0}{1:0>2}'.format(str(year), str(month-1))
                    print("데이터 없음.... \n 제공되는 통계 데이터는 %s년 %s월까지입니다." %(str(year), str(month-1)))
                    break
                # jsonData를 출력하여 확인
                print(json.dumps(jsonData, indent = 4, sort_keys = True, ensure_ascii = False))
                natName = jsonData['response']['body']['items']['item']['natKorNm']
                natName = natName.replace(' ', '')
                num = jsonData['response']['body']['items']['item']['num']
                ed = jsonData['response']['body']['items']['item']['ed']
                print('[ %s_%s : %s]' %(natName, yyyymm, num))
                print('---------------------------------------------------')
                jsonResult.append({'nat_name' : natName, 'nat_cd': nat_cd, 'yyyymm': yyyymm, 'visit_cnt': num})
                result.append([natName, nat_cd, yyyymm, num])

    return (jsonResult, result, natName, ed, dataEND)


def main():
    jsonResult = []
    result = []

    print("<< 국내 입국한 외국인의 통계 데이터를 수집합니다. >>")
    nat_cd = input("국가 코드를 입력하세요(중국: 112 / 일본: 130 / 미국: 275) : ")
    nStartYear = int(input('데이터를 몇 년부터 수집할까요? : '))
    nEndYear = int(input('데이터를 몇 년까지 수집할까요? : '))
    ed_cd = "E" # E: 방한외래관광객, D: 해외출국

    jsonResult, result, natName, ed, dataEND = getTourismStatsService(nat_cd, ed_cd, nStartYear, nEndYear)

    # 파일저장 1 : json 파일
    with open('./%s_%s_%d_%s.json' %(natName, ed, nStartYear, dataEND), 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult, indent = 4, sort_keys=True, ensure_ascii=False)
        outfile.write(jsonFile)

    # 파일저장 2 : cvs 파일
    columns = ['입국자국가', '국가코드', '입국연월', '입국자 수']
    result_df = pd.DataFrame(result, columns=columns)
    # result_df.to_csv('./%s_%s_%d_%s.csv' % (natName, ed, nStartYear, dataEND), index = False, encoding= 'cp949')
    result_df.to_csv('./%s_%s_%d_%s.csv' % (natName, ed, nStartYear, dataEND), index = False, encoding= 'utf8')

if __name__  == '__main__':
    main()

C:\Users\chaew\AppData\Local\Temp\ipykernel_47768\453859294.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


<< 국내 입국한 외국인의 통계 데이터를 수집합니다. >>
http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList?_type=json&serviceKey=Ody77GLuYeR%2FeFqbpduMN2Bi4Cka2fztbgnj6E2Eux1kUhy3e4epR28XKBUaObiqPoVzAizxXMBPXtMyuC9v9Q%3D%3D&YM=202001&NAT_CD=130&ED_CD=E
[2024-03-13 09:54:48.485848] Url Request Success
{
    "response": {
        "body": {
            "items": {
                "item": {
                    "ed": "방한외래관광객",
                    "edCd": "E",
                    "natCd": 130,
                    "natKorNm": "일  본",
                    "num": 203969,
                    "rnum": 1,
                    "ym": 202001
                }
            },
            "numOfRows": 10,
            "pageNo": 1,
            "totalCount": 1
        },
        "header": {
            "resultCode": "0000",
            "resultMsg": "OK"
        }
    }
}
[ 일본_202001 : 203969]
---------------------------------------------------
http://openapi.tour.go.kr/openapi/service/Edrc

In [4]:
nEndYear = 19700105
dataEND = "{0}{1:0>2}".format(str(nEndYear), str(12))
print(dataEND)

1970010512
